## Notebook for spike sorting from .kwd data using the Pipeline of mountainsort (linux channel has pipeline)
Uses:
    - intan2kwik (https://github.com/zekearneodo/intan2kwik/blob/master/README.md)
    - mountainlab suite(https://github.com/flatironinstitute/mountainlab-js)
    - mountainsort https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md
    - mountainsort examples https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md

In [1]:
import socket
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
from datetime import timedelta
from importlib import reload

# pipeline imports
from pipefinch.neural.convert import intan
from pipefinch.neural.sort.mountain import core as msc
from pipefinch.h5tools.kwik import kutil
from pipefinch.pipeline import probes


from pipefinch.h5tools.kwik import kwdfunctions as kwdf

from intan2kwik import kwd

#mountainsort imports (for sorting)
#import mountainlab_pytools.mlproc as mlp

import logging

# Setup the logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
logger.addHandler(ch)
        
logger.info('Logger set')
logger.info('Hostname {}'.format(socket.gethostname()))

2019-05-16 12:33:22,004 - root - INFO - Logger set
2019-05-16 12:33:22,005 - root - INFO - Hostname zpikezorter


### Session parameters and raw files

In [2]:
from pipefinch.pipeline import filestructure as et
reload(et)
reload(kwd)

sess_par = {'bird': 'b11j11',
           'sess': '2019-03-19_3125_02',
           'probe': 'probe_0', # probe to sort ('probe_0', 'probe_1') (to lookup in the rig_par which port to extract)
            'sort': 0
           }

exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], sess_par['sort'])

# mountainsort parameters
sort_params = {'adjacency_radius': -1,
              'detect_threshold': 2,
              'freq_min': 600}

# differetn mountainsort parameters
ds_params = {'detect_sign': -1}


# convenient paths
kwik_folder = exp_struct['folders']['kwik']
msort_folder = exp_struct['folders']['msort']
raw_folder = exp_struct['folders']['raw']
kwd_path = exp_struct['files']['kwd']
bin_path = exp_struct['files']['mda_raw']

In [3]:
exp_struct

{'folders': {'raw': '/mnt/microdrive/birds/b11j11/Ephys/raw/2019-03-19_3125_02',
  'kwik': '/data/experiment/microdrive/b11j11/Ephys/kwik/2019-03-19_3125_02',
  'msort': '/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02'},
 'files': {'par': '/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/params.json',
  'set': '/mnt/microdrive/birds/b11j11/Ephys/raw/2019-03-19_3125_02/settings.isf',
  'rig': '/mnt/microdrive/birds/b11j11/Ephys/raw/2019-03-19_3125_02/rig.json',
  'kwd': '/data/experiment/microdrive/b11j11/Ephys/kwik/2019-03-19_3125_02/streams.kwd',
  'kwik': '/data/experiment/microdrive/b11j11/Ephys/kwik/2019-03-19_3125_02/sort_0/spikes.kwik',
  'kwe': '/data/experiment/microdrive/b11j11/Ephys/kwik/2019-03-19_3125_02/events.kwe',
  'mda_raw': '/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/raw.mda'}}

### convert the whole session to a .kwd file
Conversion sends every .rhd file in the folder to a rec in the .kwd file (experiment.kwd in the session ss folder)
All of the files and all of the channels are converted; filtering and subselection of sub-epochs and channels occurs later.
The .kwd is raw data, only in a friendlier format.


#### Make a file for the session for the first time

In [4]:
reload(kwd)
## Convert the whole session to a kwd file
os.makedirs(kwik_folder, exist_ok=True)
first_intan_hdr, sess_pd = kwd.intan_to_kwd(raw_folder, kwd_path, overwrite=False, notempfile=True)

2019-05-16 12:33:23,088 - intan2kwik.kwd - WARNING - File /data/experiment/microdrive/b11j11/Ephys/kwik/2019-03-19_3125_02/streams.kwd already exists
2019-05-16 12:33:23,089 - intan2kwik.kwd - WARNING - Will abort. To overwrite the existing file, use overwrite=True option


RuntimeError: Kwd file already exists

#### Update a session with subsequently recorded rhd files

In [5]:
reload(kwdf)
_, nu_pd, _ = kwdf.update_kwd(kwd_path, raw_folder)

2019-05-16 12:33:24,647 - pipefinch.h5tools.kwik.kwdfunctions - INFO - updating kwd file /data/experiment/microdrive/b11j11/Ephys/kwik/2019-03-19_3125_02/streams.kwd from folder /mnt/microdrive/birds/b11j11/Ephys/raw/2019-03-19_3125_02
2019-05-16 12:33:24,915 - pipefinch.h5tools.kwik.kwdfunctions - INFO - No new files to add to the file


## Make .mda file with a set of recordings in a session
 - pick all in port A
 - get all rec within a time range


In [6]:
# get the session meta
pd_meta = kwdf.get_all_rec_meta(kwd_path)
pd_meta.head(1)


,bit_depth,name,sample_rate,start_sample,start_time,channel_bit_volts,channel_names,channels_sample_rate,dig_channel_names,is_multiSampleRate_data,valid_samples,samples_count
0,16,0,20000.0,0,2019-03-19 17:16:29,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[D-004, D-005, D-006, D-007, D-008, D-012, D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[DIN-00, DIN-01]",0,"[1177200.0, 1177200.0, 1177200.0, 1177200.0, 1...",1177200


### pick a time interval of the recordings

In [7]:
def select_time_span(meta_pd, start, span_minutes):
    end = start + timedelta(minutes=span_minutes)
    pd_selection = meta_pd.loc[meta_pd['start_time'].between(start, end)]
    return pd_selection

pd_meta_selection = select_time_span(pd_meta, pd_meta['start_time'][0]+ timedelta(minutes=0*12), 240)

In [8]:
pd_meta['start_time'].max() - pd_meta['start_time'].min()

Timedelta('0 days 14:47:06')

In [9]:
# for instance
pd_meta_selection.index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            101, 102, 103, 104, 105, 106, 107, 108, 109, 110],
           dtype='int64', length=111)

In [10]:
# load the rig parameters
rig_par_file = exp_struct['files']['rig']
with open(rig_par_file, 'r') as fp:
    rig_par = json.load(fp)

# get the probe and the port where the probe was connected
selected_probe = sess_par['probe']
probe_port = rig_par['chan']['port'][selected_probe].strip('-')

# get the channel indices of the probe's port
wanted_chans = np.array([probe_port + '-'])  # all ephys channels

chan_list = kwdf.get_all_chan_names(pd_meta_selection, chan_filt=wanted_chans)

#all_rec_list = kutil.get_rec_list(exp_struct['files']['kwd'])
selection_rec_list = np.unique(pd_meta_selection['name'])

rec_chans = pd_meta.loc[pd_meta['name'] == selection_rec_list[0], 'channel_names'].values
rec_chans_idx = kwdf.find_chan_names_idx(rec_chans[0], chan_list)

# make the mda binary file
bin_path = exp_struct['files']['mda_raw']
os.makedirs(exp_struct['folders']['msort'], exist_ok=True)
# bin_file = kwdf.kwd_to_binary(exp_struct['files']['kwd'],
#                               exp_struct['files']['mda_raw'],
#                               chan_list=chan_list,
#                               rec_list=selection_rec_list, header='mda')


## Scripts for sorting

### prep the files with their nice formats, locations and names


In [11]:
reload(msc)
file_paths, out_folder = msc.make_paths(exp_struct['folders']['msort'])
os.makedirs(exp_struct['folders']['msort'], exist_ok=True)

# make the probe file
rec_chans = pd_meta.loc[pd_meta['name']==0, 'channel_names'].values
rec_chans_idx = kwdf.find_chan_names_idx(rec_chans[0], chan_list)
probe = rig_par['probe'][selected_probe]['model']
headstage = rig_par['probe'][selected_probe]['headstage']
probe_chans = rec_chans_idx - np.min(rec_chans_idx)
# try to make a probe. If it is not possible, force adjacency_radius to -1.
try:
    probe_geom = probes.make_map(probe, probe_chans)
    np.savetxt(file_paths['geom'], probe_geom, delimiter=',')
except KeyError as err:
    logger.warning('Probe could not be made, probe or headstage not found {} in probes.py. Will sort with no geometry'.format(err))
    sort_params['adjacency_radius'] = -1

# parameters to pass to the msort scripts
ds_params.update({'samplerate': int(kwdf.get_sampling_rate(pd_meta, 0)), # required
            })


with open(file_paths['params'], 'w') as fp:
    json.dump(ds_params, fp)
    logger.info('Created session par files {}'.format(file_paths['params']))


2019-05-16 12:33:41,781 - root - WARNING - Probe could not be made, probe or headstage not found 'carbon-fiber' in probes.py. Will sort with no geometry
2019-05-16 12:33:41,783 - root - INFO - Created session par files /data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/params.json


In [12]:
file_paths

{'mda': '/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/raw.mda',
 'params': '/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/params.json',
 'geom': '/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/geom.csv',
 'filt': '/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/sort_out/filt.mda.prv',
 'pre': '/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/sort_out/pre.mda.prv',
 'firings': '/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/sort_out/firings.mda',
 'firings_curated': '/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/sort_out/firings_curated.mda',
 'cluster_metrics': '/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/sort_out/cluster_metrics.json',
 'cluster_metrics_curated': '/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/sort_out/cluster_metrics_curated.json'}

In [13]:
# test mountainsort method for reading sorting parameters
msc.read_dataset_params(exp_struct['folders']['msort'])
output_dir = os.path.join(exp_struct['folders']['msort'], 'sort_out');

{'detect_sign': -1, 'samplerate': 20000}

### Sorting with a Pipeline
If it works it's cool. If it fails, you have to disregard the Pipeline object and call msc.sort_dataset with dispatch_method='run' option)

In [16]:
from mountainlab_pytools import mlproc as mlp
Pipeline=mlp.initPipeline();

JSProxyWidget(status='Not yet rendered')

In [18]:
# Pipeline was failing, need to debug this ('NameError: name 'widgets' is not defined')

with Pipeline:
    msc.sort_dataset(file_paths=file_paths, **sort_params, dispatch_method='add')

2019-05-16 12:37:53,846 - pipefinch.sort.mountain.comre - INFO - Bandpass filter
2019-05-16 12:37:53,851 - pipefinch.sort.mountain.comre - INFO - Whitening
2019-05-16 12:37:53,855 - pipefinch.sort.mountain.comre - INFO - Sorting
/home/ezequiel/repos/pipefinch/pipefinch/neural/sort/mountain/core.py:207: UserWarning: Will sort with no geometry input
  warnings.warn('Will sort with no geometry input')
2019-05-16 12:37:53,859 - pipefinch.sort.mountain.comre - INFO - Getting cluster metrics
2019-05-16 12:37:53,869 - pipefinch.sort.mountain.comre - INFO - Automatically curating


Output()

Finished pipeline.


In [14]:
# If pipleine fails
#msc.sort_dataset(file_paths=file_paths, **sort_params,  dispatch_method='run');

2019-05-16 11:41:06,824 - pipefinch.sort.mountain.comre - INFO - Bandpass filter


RUNNING: ml-run-process ephys.bandpass_filter --inputs timeseries:/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/raw.mda --parameters freq_max:6000 freq_min:600 samplerate:20000 --outputs timeseries_out:/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/sort_out/filt.mda.prv
[ Getting processor spec... ]
This is only a warning: Unable to hard link file /data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/raw.mda -> /tmp/mountainlab-tmp/tempdir_922754fd88_5dxQaF/input_timeseries_dHtPAP4X.mda. Perhaps temporary directory is not on the same device as the output file directory. Copying instead.


2019-05-16 11:42:54,554 - pipefinch.sort.mountain.comre - INFO - Whitening


[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: 922754fd8893d25911a197178916d14793cb2286
[ Checking outputs... ]
{"timeseries_out":"/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/sort_out/filt.mda.prv"}
Processing ouput - /data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/sort_out/filt.mda.prv
false
{"timeseries_out":"/tmp/mountainlab-tmp/output_922754fd8893d25911a197178916d14793cb2286_timeseries_out.mda"}
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Creating links to input files... ]
[ Preparing temporary outputs... ]
Processing ouput - /tmp/mountainlab-tmp/output_922754fd8893d25911a197178916d14793cb2286_timeseries_out.mda
false
[ Initializing process ... ]
[ Running ... ] /home/ezequiel/anaconda3/envs/sort/bin/python3 /home/ezequiel/anaconda3/envs/sort/etc/mountainlab/packages/ml_ephys/preprocessing/preprocessing.py.mp ephys.bandpass_filter --_tempdir=/tmp/mountainlab-tmp/t

2019-05-16 11:44:45,513 - pipefinch.sort.mountain.comre - INFO - Sorting


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: 25db9f82a7ca96c8b1df78fc71ce39e656f9e805
[ Checking outputs... ]
{"timeseries_out":"/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/sort_out/pre.mda.prv"}
Processing ouput - /data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/sort_out/pre.mda.prv
false
{"timeseries_out":"/tmp/mountainlab-tmp/output_25db9f82a7ca96c8b1df78fc71ce39e656f9e805_timeseries_out.mda"}
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Creating links to input files... ]
[ Preparing temporary outputs... ]
Processing ouput - /tmp/mountainlab-tmp/output_25db9f82a7ca96c8b1df78fc71ce39e656f9e805_timeseries_out.mda
false
[ Initializing process ... ]
[ Running ... ] /home/ezequiel/anaconda3/envs/sort/bin/python3 /home/ezequiel/anaconda3/envs/sort/etc/mountainlab/packages/ml_ephys/preprocessing/preprocessing.py.mp ephys.whiten --_tempdir=/tm

/home/ezequiel/repos/pipefinch/pipefinch/neural/sort/mountain/core.py:207: UserWarning: Will sort with no geometry input
  warnings.warn('Will sort with no geometry input')


RUNNING: ml-run-process ms4alg.sort --inputs timeseries:/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/sort_out/pre.mda.prv --parameters adjacency_radius:-1 detect_sign:-1 detect_threshold:2 --outputs firings_out:/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/sort_out/firings.mda
[ Getting processor spec... ]
Traceback (most recent call last):
[ Checking inputs and substituting prvs ... ]
File "/home/ezequiel/anaconda3/envs/sort/lib/python3.6/site-packages/ml_ms4alg/ms4alg.py", line 577, in prepare_timeseries_hdf5
[ Computing process signature ... ]
f.create_dataset('part-{}-{}'.format(m,j),data=padded_chunk[m,:].ravel())
Process signature: ba8e0f2758f9006d8df958c2af3fbab8e57dfd5d
File "/home/ezequiel/anaconda3/envs/sort/lib/python3.6/site-packages/h5py/_hl/group.py", line 136, in create_dataset
[ Checking outputs... ]
dsid = dataset.make_new_dset(self, shape, dtype, data, **kwds)
{"firings_out":"/data/experiment/microdrive/b11j11/Ephys/msort/2019-

Exception: Non-zero exit code for ms4alg.sort

In [19]:
file_paths

{'mda': '/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/raw.mda',
 'params': '/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/params.json',
 'geom': '/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/geom.csv',
 'filt': '/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/sort_out/filt.mda.prv',
 'pre': '/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/sort_out/pre.mda.prv',
 'firings': '/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/sort_out/firings.mda',
 'firings_curated': '/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/sort_out/firings_curated.mda',
 'cluster_metrics': '/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/sort_out/cluster_metrics.json',
 'cluster_metrics_curated': '/data/experiment/microdrive/b11j11/Ephys/msort/2019-03-19_3125_02/sort_out/cluster_metrics_curated.json'}

## Command for viewing:
 - open up terminal with the environment msort
 - go go the ss_data folder for the session
 - run the command: qt-mountainview --raw raw.mda --filt sort_out/filt.mda.prv --pre sort_out/pre.mda.prv --samplerate=20000 --firings sort_out/firings.mda --cluster_metrics sort_out/cluster_metrics.json

# After manual curation
 - save the curated spikes in the sort_out/firings_curated.mda
 - come back to the notebook and run 

In [110]:
from pipefinch.h5tools.kwik import kwikfunctions as kwkf
reload(kwkf)
reload(et)
firings_to_save = 'firings_curated' # 'curated' or 'firings' for default_output

metrics_to_save = 'cluster_metrics_curated' if firings_to_save == 'firings_curated' else 'cluster_metrics'
kwkf.mda_to_kwik(exp_struct['files']['kwd'],
                 exp_struct['files']['kwik'],
                 file_paths[firings_to_save],
                file_paths[metrics_to_save], 
                rec_in_binary=selection_rec_list)


2019-03-20 10:28:28,647 - pipefinch.h5tools.kwik.kwikfunctions - INFO - Creating kwik file /data/experiment/microdrive/b11j11/Ephys/kwik/2019-03-15_3125_01/sort_0/spikes.kwik from firings /mnt/scratch/experiment/b11j11/Ephys/msort/2019-03-15_3125_01/sort_out/firings_curated.mda


In [111]:
selection_rec_list

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44])

In [112]:
### extract all unit waveforms
from pipefinch.neural import units
reload(units)
units.get_all_unit_waveforms(exp_struct['files']['kwik'], exp_struct['files']['kwd'], port=probe_port)

2019-03-20 10:28:39,775 - pipefinch.neural.units - INFO - About to get all waveforms for 29 units in file /data/experiment/microdrive/b11j11/Ephys/kwik/2019-03-15_3125_01/sort_0/spikes.kwik


/home/zeke/anaconda3/envs/mountain/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


0

#### cleanup the intermediate sort files and the mountainsort .mda files

In [113]:
reload(et)
reload(msc)
et.msort_cleanup(exp_struct)
msc.clean_tmp_dir()

2019-03-20 10:30:05,875 - pipefinch.pipeline.filestructure - INFO - removing (if exists) msort mda file /mnt/scratch/experiment/b11j11/Ephys/msort/2019-03-15_3125_01/raw.mda 
2019-03-20 10:30:06,602 - pipefinch.sort.mountain.comre - INFO - Cleaning up msort temp dir /mnt/scratch/experiment/mountainlab-tmp


In [114]:
 exp_struct['files']

{'par': '/mnt/scratch/experiment/b11j11/Ephys/msort/2019-03-15_3125_01/params.json',
 'set': '/mnt/zuperfinch/microdrive/birds/b11j11/Ephys/raw/2019-03-15_3125_01/settings.isf',
 'rig': '/mnt/zuperfinch/microdrive/birds/b11j11/Ephys/raw/2019-03-15_3125_01/rig.json',
 'kwd': '/data/experiment/microdrive/b11j11/Ephys/kwik/2019-03-15_3125_01/streams.kwd',
 'kwik': '/data/experiment/microdrive/b11j11/Ephys/kwik/2019-03-15_3125_01/sort_0/spikes.kwik',
 'kwe': '/data/experiment/microdrive/b11j11/Ephys/kwik/2019-03-15_3125_01/events.kwe',
 'mda_raw': '/mnt/scratch/experiment/b11j11/Ephys/msort/2019-03-15_3125_01/raw.mda'}